$\newcommand{\xbf}{{\bf x}}
\newcommand{\ybf}{{\bf y}}
\newcommand{\wbf}{{\bf w}}
\newcommand{\Ibf}{\mathbf{I}}
\newcommand{\Xbf}{\mathbf{X}}
\newcommand{\Rbb}{\mathbb{R}}
\newcommand{\vec}[1]{\left[\begin{array}{c}#1\end{array}\right]}
$

# Les filtres convolutifs (partie 2)
Matériel de cours rédigé par Pascal Germain, 2019
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
import torch
from torch import nn
torch.__version__ # Ce notebook a été conçu avec la version '1.2.0' de pytorch

### Chargement des données «MNIST» et pré-traitements

In [ ]:
repertoire_mnist = '../data/mnist/' # Modifier le répertoire au besoin

In [ ]:
def charger_mnist(repertoire, etiquettes=None, max_par_etiquettes=None):
    if etiquettes is None:
         etiquettes = np.arange(10)
    images_list = [None] * len(etiquettes)
    labels_list = [None] * len(etiquettes)
    for i, val in enumerate(etiquettes):
        nom_fichier = repertoire + f'mnist_{val}.gz'
        images_list[i] = np.genfromtxt(nom_fichier, max_rows=max_par_etiquettes, dtype=np.float32)
        nb = images_list[i].shape[0]

        labels_list[i] = i*np.ones(nb, dtype=np.int64)
        print(val, ':', nb, 'images')
        
    x = np.vstack(images_list)
    y = np.concatenate(labels_list)
    print('Total :', len(y), 'images')
    return x, y

In [ ]:
data_x, data_y = charger_mnist(repertoire_mnist, etiquettes=None, max_par_etiquettes=1000)
data_x = data_x / 255

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.5, random_state=42)
print('train_x:', train_x.shape)
print('test_x:', test_x.shape)
print('train_y:', train_y.shape)
print('test_y:', test_y.shape)

In [ ]:
plt.figure(figsize=(15,2))
plt.imshow(train_x, cmap=plt.cm.gray, aspect=.025)
plt.colorbar()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_x)
train_x_prime = scaler.transform(train_x)
test_x_prime = scaler.transform(test_x)

In [ ]:
plt.figure(figsize=(15,2))
plt.imshow(train_x_prime, vmin=-1.5, vmax=1.5, cmap=plt.cm.gray, aspect=.025)
plt.colorbar()

### Créons un réseau de neurones

...

In [ ]:
from reseau_classif_generique import ReseauClassifGenerique

In [ ]:
ReseauClassifGenerique?

...

In [ ]:
class UneArchiPourMNIST(nn.Module):
    def __init__(self, nb_filtres=32, taille_noyau=3):
        # Initialisation de la classe de base nn.Module
        super().__init__()
        
        # Créons une couche de convolution 
        self.modele_conv = nn.Sequential(
            nn.Conv2d(1, nb_filtres, kernel_size=taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche de sortie 
        self.nb_neurones_du_milieu = nb_filtres * ((28-taille_noyau+1)//2)**2
        self.modele_plein = nn.Sequential(
            nn.Linear(self.nb_neurones_du_milieu, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x, apprentissage=False):
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3


In [ ]:
mon_archi = UneArchiPourMNIST(nb_filtres=1, taille_noyau=5)
mon_archi

In [ ]:
for params in mon_archi.parameters():
    print(params)
    print('----')

In [ ]:
une_image = torch.tensor(train_x_prime[1,:])
mon_archi(une_image)

Nous pouvons maintenant faire apprendre notre réseau convolutif

In [ ]:
mon_archi = UneArchiPourMNIST(nb_filtres=32, taille_noyau=3)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)

In [ ]:
R.apprentissage(train_x_prime, train_y)

In [ ]:
from sklearn.metrics import accuracy_score
train_pred = R.prediction(train_x_prime)
test_pred = R.prediction(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

### Visualisation des filtres appris

In [ ]:
from torchvision.utils import make_grid
def afficher_grille(images):
    plt.figure(figsize=(15,4))
    grid = make_grid(images, pad_value=torch.max(images).item())
    plt.imshow(grid[0].detach(), cmap=plt.cm.gray)
    plt.colorbar()

In [ ]:
afficher_grille(mon_archi.modele_conv[0].weight)

### Visualisation de la représentation interne du réseau

In [ ]:
def afficher_activations(x, modele, etape):
    image = torch.tensor(x).view(1,1,28,28)
    sous_modele = modele[0:etape]
    print(sous_modele)
    couche = sous_modele(image)
    afficher_grille(couche.transpose(0,1))

In [ ]:
afficher_activations(train_x_prime[0], mon_archi.modele_conv, 1)

In [ ]:
afficher_activations(train_x_prime[0], mon_archi.modele_conv, 2)

In [ ]:
afficher_activations(train_x_prime[0], mon_archi.modele_conv, 3)

************

# À vous de jouer.

Essayez d'autres architectures de réseau de neurones convolutif, et tentez d'interpréter les résultats obtenus. Parmi les différentes possibilités, vous pouvez:
* Changer la taille et le nombre de filtres convolutifs
* Ajouter une ou plusieurs couches de filtres convolutifs dans la première partie du réseau
* Ajouter davantage de couches pleinement connectées dans la seconde partie du réseau
* Comparer les résultats obtenus sans appliquer la renormalisation `StandardScaler`


In [ ]:
mon_archi = UneArchiPourMNIST(nb_filtres=64, taille_noyau=9)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)
R.apprentissage(train_x_prime, train_y)

In [ ]:
train_pred = R.prediction(train_x_prime)
test_pred = R.prediction(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

In [ ]:
class Archi2CouchesCovolutives(nn.Module):
    def __init__(self, couche1_nb_filtres=16, couche1_taille_noyau=9, 
                       couche2_nb_filtres=32, couche2_taille_noyau=3):
        # Initialisation de la classe de base nn.Module
        super().__init__()
        
        # Créons deux couches de convolution 
        self.modele_conv = nn.Sequential(
            # Première couche de convolution
            nn.Conv2d(1, couche1_nb_filtres, kernel_size=couche1_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # NOUVEAUTÉ: Deuxième couche de convolution
            nn.Conv2d(couche1_nb_filtres, couche2_nb_filtres, kernel_size=couche2_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche de sortie
        # NOUVEAUTÉ: Calcul du nombre de neurones sur la couche pleinement connectée
        #            considérant la 2e couche de convolution
        nb_pixels = (28 - couche1_taille_noyau + 1) // 2 
        nb_pixels = (nb_pixels - couche2_taille_noyau + 1) // 2
        self.nb_neurones_du_milieu = couche2_nb_filtres * (nb_pixels**2)
        
        self.modele_plein = nn.Sequential(
            nn.Linear(self.nb_neurones_du_milieu, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3
    

In [ ]:
mon_archi = Archi2CouchesCovolutives(couche1_nb_filtres=16, couche1_taille_noyau=9,
                                     couche2_nb_filtres=32, couche2_taille_noyau=3)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)
R.apprentissage(train_x_prime, train_y)
train_pred = R.prediction(train_x_prime)
test_pred = R.prediction(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

In [ ]:
class Archi2CouchesCovolutives2CouchesPC(nn.Module):
    def __init__(self, couche1_nb_filtres=16, couche1_taille_noyau=9, 
                       couche2_nb_filtres=32, couche2_taille_noyau=3,
                       couche3_nb_neurones=100):
        # Initialisation de la classe de base nn.Module
        super().__init__()
        
        # Créons deux couches de convolution 
        self.modele_conv = nn.Sequential(
            # Première couche de convolution
            nn.Conv2d(1, couche1_nb_filtres, kernel_size=couche1_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # Deuxième couche de convolution
            nn.Conv2d(couche1_nb_filtres, couche2_nb_filtres, kernel_size=couche2_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche cachée pleinement connectée
        nb_pixels = (28 - couche1_taille_noyau + 1) // 2
        nb_pixels = (nb_pixels - couche2_taille_noyau + 1) // 2
        self.nb_neurones_du_milieu = couche2_nb_filtres * (nb_pixels**2)
        
        self.modele_plein = nn.Sequential(
            # NOUVEAUTÉ: Nouvelle couche cachée pleinement connectée avec activation ReLU
            nn.Linear(self.nb_neurones_du_milieu, couche3_nb_neurones),  
            nn.ReLU(),
            # Couche de sortie
            nn.Linear(couche3_nb_neurones, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):         
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3

In [ ]:
mon_archi = Archi2CouchesCovolutives2CouchesPC(couche1_nb_filtres=16, couche1_taille_noyau=9,
                                               couche2_nb_filtres=32, couche2_taille_noyau=3,
                                               couche3_nb_neurones=100)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)
R.apprentissage(train_x_prime, train_y)
train_pred = R.prediction(train_x_prime)
test_pred = R.prediction(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))